In [8]:
!pip install transformers datasets accelerate -q

In [2]:
!pip install torch seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import TrainingArguments, Trainer

In [4]:
dataset = load_dataset("squad")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [6]:
dataset['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [7]:
model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

BertForQuestionAnswering LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
bert.pooler.dense.weight                   | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
bert.pooler.dense.bias                     | UNEXPECTED | 
qa_outputs.bias                            | MISSING    | 
qa_outputs.weight                          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized beca

In [9]:
def preprocess_function(examples):
  questions = [q.strip() for q in examples['question']]  # remove leading/trailing spaces from each question.

  # tokenize: (question, context) -> BERT format: [CLS] question [SEP] context [SEP]
  inputs = tokenizer(
      questions,
      examples['context'],
      max_length=384,
      truncation='only_second', # if too long, truncate context only; never truncate question
      padding='max_length', # all sequences to length 384
      return_offsets_mapping=True # this return [(start_char, end_char), ...] for each token; EX-: Token: "California" , offset:(25, 35)
  )

  # remove offset mapping from inputs
  # model does not need it, only need for computing start/end positions
  offset_mapping = inputs.pop('offset_mapping')
  answers = examples['answers']
  start_positions = []
  end_positions = []

  for i, offsets in enumerate(offset_mapping):  # loop over batch, offset: list of start_char, end_char for each token
    answer = answers[i]
    start_char = answer['answer_start'][0]  # start of answer in raw context
    end_char = start_char + len(answer['text'][0]) # end of answer
    sequence_ids = inputs.sequence_ids(i) # identify which token belong to context; None-Special token, 0-question, 1-context
    # EX: [None, 0, 0, 0, None, 1, 1, 1, 1, None]

    # find context start
    idx = 0
    while sequence_ids[idx] != 1: # move untill reach first token with sequence_id=1
      idx += 1
    context_start = idx

    # find context end
    while sequence_ids[idx] == 1:
      idx += 1
    context_end = idx - 1

    # context token = context_start -> context_end

    # check is the answer span fully inside the context token? , this handled the impossible answers
    if not (offsets[context_start][0] <= start_char and offsets[context_end][1] >= end_char):
      start_positions.append(0)
      end_positions.append(0)
    else:
      # find start token index
      idx = context_start
      while idx <= context_end and offsets[idx][0] <= start_char:  # move through context context tokens untill token_start_char > answer_start_char
        idx += 1
      start_positions.append(idx - 1)

      # find end token index
      idx = context_end
      while idx >= context_start and offsets[idx][1] >= end_char:  # move backwards untill token_end_char < answer_end_char
        idx -= 1
      end_positions.append(idx + 1)

  inputs['start_positions'] = start_positions
  inputs['end_positions'] = end_positions

  # model receives: input_ids, attention_mask, start_positions, end_positions

  return inputs

In [10]:
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['train'].column_names
    # after preprocessing function will returns input_ids, attention_mask, start_positions, end_positions so remove old column names
)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [11]:
training_args = TrainingArguments(
    output_dir="./qa_results",  # this will save model checkpoints, final trained model, training states
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",  # save training logs
)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


In [14]:
from transformers import DataCollatorWithPadding
# collator dynamically padding inputs in a bacth
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)  # pads all the sequences to longest sequence in the batch

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].select(range(2000)),
    eval_dataset=tokenized_datasets["validation"].select(range(500)),
    data_collator=data_collator,
)

In [15]:
trainer.train()

Step,Training Loss


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=250, training_loss=3.885894775390625, metrics={'train_runtime': 164.0533, 'train_samples_per_second': 12.191, 'train_steps_per_second': 1.524, 'total_flos': 391945135104000.0, 'train_loss': 3.885894775390625, 'epoch': 1.0})

In [16]:
trainer.evaluate()

{'eval_loss': 2.7218618392944336,
 'eval_runtime': 12.9827,
 'eval_samples_per_second': 38.513,
 'eval_steps_per_second': 4.853,
 'epoch': 1.0}

In [18]:
context = "Transformers are deep learning models that use self-attention to understand language."
question = "What do transformers use to understand language?"

inputs = tokenizer(question, context, return_tensors="pt") # return PyTorch tensors

# Move inputs to the same device as the model
inputs = {k: v.to(model.device) for k, v in inputs.items()} # ensure everthing is on same device (CPU or CUDA)

outputs = model(**inputs)

start = torch.argmax(outputs.start_logits) # finds token index with highest start probability
end = torch.argmax(outputs.end_logits) + 1  # adding +1 ensures correct span

# Move input_ids back to CPU for token conversion if the model was on GPU
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(inputs["input_ids"][0].cpu()[start:end])
)

print("Answer:", answer)

Answer: deep learning models that use self - attention to understand language
